## Load libraries

In [ ]:
import os

from dotenv import load_dotenv
load_dotenv()

# CONST

CHUSER = os.getenv('CHUSER')
CHPASS = os.getenv('CHPASS')



## Spark initialize

In [ ]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf, SQLContext

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os
#https://repo1.maven.org/maven2/com/github/housepower/clickhouse-native-jdbc/2.7.1/clickhouse-native-jdbc-2.7.1.jar
packages = [
    "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3",
    "com.clickhouse:clickhouse-jdbc:0.6.0",
    "com.clickhouse:clickhouse-http-client:0.6.0",
    "org.apache.httpcomponents.client5:httpclient5:5.3.1",
    "com.github.housepower:clickhouse-native-jdbc:2.7.1"
]
ram = 30
cpu = 22*3
# Define the application name and setup session
appName = "Connect To ClickHouse via PySpark"
spark = (SparkSession.builder
         .appName(appName)
         .config("spark.jars.packages", "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3,com.clickhouse:clickhouse-jdbc:0.6.0,com.clickhouse:clickhouse-http-client:0.6.0,org.apache.httpcomponents.client5:httpclient5:5.3.1,com.github.housepower:clickhouse-native-jdbc:2.7.1")
         .config("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
         .config("spark.sql.catalog.clickhouse.host", "35.217.7.77")
         .config("spark.sql.catalog.clickhouse.protocol", "http")
         .config("spark.sql.catalog.clickhouse.http_port", "8123")
         .config("spark.sql.catalog.clickhouse.user", "default")
         .config("spark.sql.catalog.clickhouse.password", "Gabadura123$")
         .config("spark.sql.catalog.clickhouse.database", "default")
         .config("spark.spark.clickhouse.write.compression.codec", "lz4")
         .config("spark.clickhouse.read.compression.codec", "lz4")
         .config("spark.clickhouse.write.format", "arrow")
         #    .config("spark.clickhouse.write.distributed.convertLocal", "true") 
         #    .config("spark.clickhouse.write.repartitionNum", "1") 
         #.config("spark.clickhouse.write.maxRetry", "1000")
         #    .config("spark.clickhouse.write.repartitionStrictly", "true") 
         #    .config("spark.clickhouse.write.distributed.useClusterNodes", "false") 
         .config("spark.clickhouse.write.batchSize", "1000000")
         #.config("spark.sql.catalog.clickhouse.socket_timeout", "600000000")
        #  .config("spark.sql.catalog.clickhouse.connection_timeout", "600000000")
        #  .config("spark.sql.catalog.clickhouse.query_timeout", "600000000")
        #  .config("spark.clickhouse.options.socket_timeout", "600000000")
        #  .config("spark.clickhouse.options.connection_timeout", "600000000")
        #  .config("spark.clickhouse.options.query_timeout", "600000000")         
         .config("spark.executor.memory", f"{ram}g")
         .config("spark.executor.cores", "5")
         .config("spark.driver.maxResultSize", f"{ram}g")
         .config("spark.driver.memory", f"{ram}g")
         .config("spark.executor.memoryOverhead", f"{ram}g")
         .config("spark.sql.debug.maxToStringFields", "100000")
         .getOrCreate()
         )
#SedonaRegistrator.registerAll(spark)
# spark.conf.set("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
# spark.conf.set("spark.sql.catalog.clickhouse.host", "127.0.0.1")
# spark.conf.set("spark.sql.catalog.clickhouse.protocol", "http")
# spark.conf.set("spark.sql.catalog.clickhouse.http_port", "8123")
# spark.conf.set("spark.sql.catalog.clickhouse.user", "default")
# spark.conf.set("spark.sql.catalog.clickhouse.password", "")
# spark.conf.set("spark.sql.catalog.clickhouse.database", "default")
# spark.sql("use clickhouse")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.github.housepower#clickhouse-spark-runtime-3.4_2.12 added as a dependency
com.clickhouse#clickhouse-jdbc added as a dependency
com.clickhouse#clickhouse-http-client added as a dependency
org.apache.httpcomponents.client5#httpclient5 added as a dependency
com.github.housepower#clickhouse-native-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-51a114ad-5ea3-40b2-ad21-5a58e5b80e3c;1.0
	confs: [default]
	found com.github.housepower#clickhouse-spark-runtime-3.4_2.12;0.7.3 in central
	found com.clickhouse#clickhouse-jdbc;0.6.0 in central
	found com.clickhouse#clickhouse-http-client;0.6.0 in central
	found com.clickhouse#clickhouse-client;0.6.0 in central
	found com.clickhouse#clickhouse-data;0.6.0 in central
	found org.apache.httpcomponents.client5#httpclient5;5.3.1 in central
	found org.apache.httpcomponents.core5#httpcore5;5.2.4 in central
	found

In [3]:
spark.sql("use clickhouse")

DataFrame[]

In [5]:
spark.sql("show tables").show()

+---------+-----------------+-----------+
|namespace|        tableName|isTemporary|
+---------+-----------------+-----------+
|  default|        cargo_tmp|      false|
|  default|leg_price_convert|      false|
+---------+-----------------+-----------+



In [6]:
spark.sql('''insert into default.cargo_tmp select * from transerv_cont.cargo''')

24/04/28 12:18:48 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)
java.lang.NoSuchMethodError: 'org.apache.arrow.vector.types.pojo.Schema org.apache.spark.sql.util.ArrowUtils$.toArrowSchema(org.apache.spark.sql.types.StructType, java.lang.String)'
	at org.apache.spark.sql.clickhouse.SparkUtils$.toArrowSchema(SparkUtils.scala:28)
	at xenon.clickhouse.write.format.ClickHouseArrowStreamWriter.<init>(ClickHouseArrowStreamWriter.scala:31)
	at xenon.clickhouse.write.ClickHouseBatchWrite.createWriter(ClickHouseWrite.scala:73)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.run(WriteToDataSourceV2Exec.scala:436)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.run$(WriteToDataSourceV2Exec.scala:425)
	at org.apache.spark.sql.execution.datasources.v2.DataWritingSparkTask$.run(WriteToDataSourceV2Exec.scala:491)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.$anonfun$writeWithV2$2(WriteToDataSourceV2Exec.scala:388)
	at org.apache.

Py4JJavaError: An error occurred while calling o63.sql.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (04394c1e478a executor driver): java.lang.NoSuchMethodError: 'org.apache.arrow.vector.types.pojo.Schema org.apache.spark.sql.util.ArrowUtils$.toArrowSchema(org.apache.spark.sql.types.StructType, java.lang.String)'
	at org.apache.spark.sql.clickhouse.SparkUtils$.toArrowSchema(SparkUtils.scala:28)
	at xenon.clickhouse.write.format.ClickHouseArrowStreamWriter.<init>(ClickHouseArrowStreamWriter.scala:31)
	at xenon.clickhouse.write.ClickHouseBatchWrite.createWriter(ClickHouseWrite.scala:73)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.run(WriteToDataSourceV2Exec.scala:436)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.run$(WriteToDataSourceV2Exec.scala:425)
	at org.apache.spark.sql.execution.datasources.v2.DataWritingSparkTask$.run(Wri

In [ ]:
# for loading and uploading big data


(       
df    
.write
.mode("append")
.format("jdbc")
.option("url", "jdbc:clickhouse://id:9000/geo")
        .option("driver", "com.github.housepower.jdbc.ClickHouseDriver")
.option("user", "default")
.option("password", "")
.option("dbtable", "mnsi_spain1")
.option("batchsize", "10000000")
.option("numPartitions", cpu)
.option("truncate", "true")
.option("isolationLevel", "NONE")
.save()
)


test_df = (spark.read.format("jdbc")
    .option("driver", "com.github.housepower.jdbc.ClickHouseDriver")
    .option("url", "jdbc:clickhouse://id:9000")
    .option("user", "default")
    .option("password", "")
    .option("dbtable", "geo.mnsi_spain")
    .load()
 )

In [ ]:
test_df = (spark.read.format("jdbc")
    .option("driver", "com.github.housepower.jdbc.ClickHouseDriver")
    .option("url", "jdbc:clickhouse://35.217.7.77:9000")
    .option("user", "default")
    .option("password", "Gabadura123$")
    .option("dbtable", "transerv_cont.cargo")
    .load()
 )

IllegalArgumentException: Invalid offset or length (25, 51) in array of length 51